# QIIME 2 Workflow for Characterization of the aerodigestive tract microbiota in mechanically ventilated children – a pilot study:
Built on the existing workflow - Qiime2_MicrobialMatt_August2020 python notebook which was generated from LangilleLab workflow and Qiime Tutorials

August/Sept 2020

##### The purpose of this notebook is to complete all the upstream processing of the data form raw form to processed form for downstream analyses such as taxonomic classification and diversity analyses.

## Qiime 2 workflow for raw sequence processing

The samples were processed and sequenced at the Forsyth Institute. The V3-V4 region of the 16S rRNA gene from samples was sequenced according to the protocol described by Caporaso and colleagues (2011). The universal bacterial primers 341F 5’-CCTACGGGAGGCAGCAG-3’ (Muyzer et al., 1993) and 806R 5’-GGACTACHVGGGTWTCTAAT-3 (Caporaso et al., 2011) were used. 

Qiime2 uses two different types of files that contain the data and metadata for the analysis (.qza and .qzv files). To see what type of data is contained in a data file, use the command qiime tools peek filename.qza. The files will contain basic info (name, universally unique identifier, data type and dataformat). the raw data in these files can be accessed using the command qiime tools export


In [7]:
'''Activate qiime env'''
# conda activate qiime2-2020.6 prior to opening jupyter notebook

!qiime --version

q2cli version 2020.6.0
Run `qiime info` for more version details.


### “EMP protocol” multiplexed paired-end fastq

Format description

one forward.fastq.gz file that contains the forward sequence reads,

one reverse.fastq.gz file that contains the reverse sequence reads,

one barcodes.fastq.gz file that contains the associated barcode reads

In [13]:
'''Importing raw sequence files based on "EMP protocol" \
multiplexed paired-end fastq'''

# NileshMeta_Undetermined_S0_L001_R1_001.fastq.gz --> forward.fastq.gz
# NileshMeta_Undetermined_S0_L001_R2_001.fastq.gz --> reverse.fastq.gz
# NileshMehta_Undetermined_S0_L001_I1_001.fastq.gz --> barcodes.fastq.gz


!qiime tools import \
   --type EMPPairedEndSequences \
   --input-path CHBoral_PairedEndSeq_rawdata/ \
   --output-path CHBoral_PairedEndSeq.qza
   

Imported CHBoral_PairedEndSeq_rawdata/ as EMPPairedEndDirFmt to CHBoral_PairedEndSeq.qza


## Demultiplex:
#### We must demultiplex these reads to determine which sample each read came from.

Demultiplexed using demux emp-paired using reverse primer sequence that has the barcode embedded.

In [4]:
'''Demultiplex paired end sequences'''
# metadata file was renamed and reformated from SampleInfo_Nilesh Mehta 2Feb2015.xlsx

!qiime demux emp-paired \
  --m-barcodes-file mappingfile_NileshMehta_with_Metadata_062715_corrected.txt \
  --m-barcodes-column BarcodeSequence \
  --p-rev-comp-mapping-barcodes True \
  --i-seqs CHBoral_PairedEndSeq.qza \
  --o-per-sample-sequences CHBoral_PairedEndSeq_demux-full.qza \
  --o-error-correction-details CHBoral_PairedEndSeq_demux-details.qza




Saved SampleData[PairedEndSequencesWithQuality] to: CHBoral_PairedEndSeq_demux-full.qza
Saved ErrorCorrectionDetails to: CHBoral_PairedEndSeq_demux-details.qza


In [6]:
'''Summarize demultiplexed outputs'''

!qiime demux summarize \
--i-data CHBoral_PairedEndSeq_demux-full.qza \
--o-visualization CHBoral_PairedEndSeq_demux-full_qualities.qzv


# !qiime demux summarize \
# --i-data demux-full.qza \
# --o-visualization qualities.qzv

Saved Visualization to: CHBoral_PairedEndSeq_demux-full_qualities.qzv


In [18]:
'''View summary of demultiplexed qualities'''

!qiime tools view CHBoral_PairedEndSeq_demux-full_qualities.qzv


Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

#### Based on fastq stats all sequences are of >= 248bp this should allow for DADA2 run to be successful. This would provide ~31bp overlap between the 314F and 806R reads.

However, given how slow DADA2 will run and that it overlap region of at least 12bp between the forward and reverse reads, we will subsample to ensure its success. 

In [10]:
"Subsample to ensure DADA2 run is succesful"

# will result in ~10,000 seqs per sample

!qiime demux subsample-paired \
  --i-sequences CHBoral_PairedEndSeq_demux-full.qza \
  --p-fraction 0.08 \
  --o-subsampled-sequences CHBoral_PairedEndSeq_demux-subsample.qza


Saved SampleData[PairedEndSequencesWithQuality] to: CHBoral_PairedEndSeq_demux-subsample.qza


In [11]:
'''Summarize subsampled demultiplexed qualities'''

!qiime demux summarize \
  --i-data CHBoral_PairedEndSeq_demux-subsample.qza \
  --o-visualization CHBoral_PairedEndSeq_demux-subsample.qzv

Saved Visualization to: CHBoral_PairedEndSeq_demux-subsample.qzv


In [63]:
'''View summary of subsampled demultiplexed qualities'''

!qiime tools view CHBoral_PairedEndSeq_demux-subsample.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

#### Given that all sequences are of >= 248bp this should allow for DADA2 run to be successful. This would provide ~31bp overlap between the 314F and 806R reads.

### Running DADA2 workflow which does the following:
1. filter and trim the reads
2. find the most likely original sequences in the sample (ASVs)
3. remove chimeras:
--p-chimera-method 'consensus': Chimeras are detected in samples individually, and sequences found chimeric in a sufficient fraction of samples are removed.  
4. count the abundances

Full length of the reads (>=~248bp) will give ~31 bp overlap 

For quality trimming in DADA2, a read length of ~>=240bp must be maintained to get a 15bp overlap. DADA2 requires at least 12 bp overlap over it will not work.

Primer 806R is 20bp and primer 341F is 17bp. DADA2 expects primer-free reads and hence 20bp will be trimmed for forward and reverse reads. Higher number of bp is trimmed to reduce inaccuracies for ASV/OTUs.
However, given that the quality of the first ~20 bp for forward and reverse reads is very high, I will trim the first 5bp only.

In [21]:
#!qiime dada2 denoise-paired --i-demultiplexed-seqs reads_qza/reads_trimmed.qza \
 #                          --p-trunc-len-f 270 \
  #                         --p-trunc-len-r 210 \
   #                        --p-max-ee-f 2 \
    #                       --p-max-ee-r 3 \
     #                      --p-n-threads 4 \
      #                     --output-dir dada2_output --verbose


!qiime dada2 denoise-paired \
  --i-demultiplexed-seqs CHBoral_PairedEndSeq_demux-subsample.qza \
  --p-trim-left-f 5 \
  --p-trim-left-r 5 \
  --p-trunc-len-f 240 \
  --p-trunc-len-r 240 \
  --o-table DADA2_subsample/CHBoral_PairedEndSeq_demux-subsample-table.qza \
  --o-representative-sequences DADA2_subsample/CHBoral_PairedEndSeq_demux-subsample-rep-seqs.qza \
  --o-denoising-stats DADA2_subsample/CHBoral_PairedEndSeq_demux-subsample-denoising-stats.qza

Saved FeatureTable[Frequency] to: CHBoral_PairedEndSeq_demux-subsample-table.qza
Saved FeatureData[Sequence] to: CHBoral_PairedEndSeq_demux-subsample-rep-seqs.qza
Saved SampleData[DADA2Stats] to: CHBoral_PairedEndSeq_demux-subsample-denoising-stats.qza


In [39]:
'''Generate summaries for DADA2 output'''

!qiime feature-table summarize \
  --i-table DADA2_subsample/CHBoral_PairedEndSeq_demux-subsample-table.qza \
  --o-visualization DADA2_subsample/CHBoral_PairedEndSeq_demux-subsample-table.qzv \
  --m-sample-metadata-file mappingfile_NileshMehta_with_Metadata_062715_corrected.txt

!qiime feature-table tabulate-seqs \
  --i-data DADA2_subsample/CHBoral_PairedEndSeq_demux-subsample-rep-seqs.qza \
  --o-visualization DADA2_subsample/CHBoral_PairedEndSeq_demux-subsample-rep-seqs.qzv

!qiime metadata tabulate \
  --m-input-file DADA2_subsample/CHBoral_PairedEndSeq_demux-subsample-denoising-stats.qza \
  --o-visualization DADA2_subsample/CHBoral_PairedEndSeq_demux-subsample-denoising-stats.qzv


Saved Visualization to: DADA2_subsample/CHBoral_PairedEndSeq_demux-subsample-table.qzv
Saved Visualization to: DADA2_subsample/CHBoral_PairedEndSeq_demux-subsample-rep-seqs.qzv
Saved Visualization to: DADA2_subsample/CHBoral_PairedEndSeq_demux-subsample-denoising-stats.qzv


In [25]:
'''View dada2 stats for subsampled demultiplexed seqs'''

!qiime tools view CHBoral_PairedEndSeq_demux-subsample-denoising-stats.qzv



Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

#### Given that DADA2 was successful in execution for subsampled demultiplexed sequences, will not move forward with all the sequences.

In [33]:
'''Run DADA2 with same parameters as used for the subsampled seq'''

!qiime dada2 denoise-paired \
  --i-demultiplexed-seqs CHBoral_PairedEndSeq_demux-full.qza \
  --p-trim-left-f 5 \
  --p-trim-left-r 5 \
  --p-trunc-len-f 240 \
  --p-trunc-len-r 240 \
  --o-table DADA2_full/CHBoral_PairedEndSeq_demux-full-table.qza \
  --o-representative-sequences DADA2_full/CHBoral_PairedEndSeq_demux-full-rep-seqs.qza \
  --o-denoising-stats DADA2_full/CHBoral_PairedEndSeq_demux-full-denoising-stats.qza


Saved FeatureTable[Frequency] to: DADA2_full/CHBoral_PairedEndSeq_demux-full-table.qza
Saved FeatureData[Sequence] to: DADA2_full/CHBoral_PairedEndSeq_demux-full-rep-seqs.qza
Saved SampleData[DADA2Stats] to: DADA2_full/CHBoral_PairedEndSeq_demux-full-denoising-stats.qza


In [38]:
'''Generate summaries for DADA2 output'''

# Generates table with feature frequency per sample and frequency per feature
!qiime feature-table summarize \
  --i-table DADA2_full/CHBoral_PairedEndSeq_demux-full-table.qza \
  --o-visualization DADA2_full/CHBoral_PairedEndSeq_demux-full-table.qzv \
  --m-sample-metadata-file mappingfile_NileshMehta_with_Metadata_062715_corrected.txt

# Generates Sequence Length Statistics and Sequence Table
!qiime feature-table tabulate-seqs \
  --i-data DADA2_full/CHBoral_PairedEndSeq_demux-full-rep-seqs.qza \
  --o-visualization DADA2_full/CHBoral_PairedEndSeq_demux-full-rep-seqs.qzv

# Generates sequence numbers given DADA2 sequential processing table 
!qiime metadata tabulate \
  --m-input-file DADA2_full/CHBoral_PairedEndSeq_demux-full-denoising-stats.qza \
  --o-visualization DADA2_full/CHBoral_PairedEndSeq_demux-full-denoising-stats.qzv


Saved Visualization to: DADA2_full/CHBoral_PairedEndSeq_demux-full-table.qzv
Saved Visualization to: DADA2_full/CHBoral_PairedEndSeq_demux-full-rep-seqs.qzv
Saved Visualization to: DADA2_full/CHBoral_PairedEndSeq_demux-full-denoising-stats.qzv


In [14]:
'''View dada2 stats for demultiplexed seqs'''

# !qiime tools view DADA2_full/CHBoral_PairedEndSeq_demux-full-table.qzv

# !qiime tools view DADA2_full/CHBoral_PairedEndSeq_demux-full-rep-seqs.qzv

# ! qiime tools view DADA2_full/CHBoral_PairedEndSeq_demux-full-denoising-stats.qzv


Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Overall, after the DADA2 processing, average of ~45% of the sequences were retained per sample. Given the tutorials output with DADA2, our data did pretty well. This means that on average the samples have 61000 read pairs (min is ~13000 pairs and max is ~171000).

### Filter features based on sequence length

Filter features based on short sequence length: spot checked sequences by blasting short length sequences. This yielded irrelevant taxonomic assignments (i.e. human sequences). Filter out features with sequence length less than 390. 

To determine whether filtering based on feature frequency should be done, we will assign taxonomy first. Based on taxonomic classification of the rare ASVs, we can determine whether biologically speaking these features should be excluded or not.

At this point, o filtering of features based on frequency will be performed. Rare ASVs (i.e. based on illumina MiSeq error rate of 0.1%) are those with features with frequency less than 0.001 of teh mean of all the feature frequencies. 

In the case of this of data, any features less than (2,480.3201483312732* 0.001) would be considered as rare ASVs. This means that features that occur less than 2.4 times (i.e. 53 out of 809 features) will be considered as rare ASVs. This is 6% of the features, which could have a huge impact on downstream data interpretations (pariticularly for diversity analyses).

In [64]:
'''Filter for features with sequence length >390'''

!qiime feature-table filter-seqs \
    --i-data DADA2_full/CHBoral_PairedEndSeq_demux-full-rep-seqs.qza \
    --m-metadata-file DADA2_full/CHBoral_PairedEndSeq_demux-full-rep-seqs.qza \
    --p-where 'length(sequence) > 390' \
    --o-filtered-data DADA2_full/CHBoral_PairedEndSeq_demux-full-L390-filtered-seqs.qza

'''View new feature table for filtered seqs'''

!qiime feature-table tabulate-seqs \
  --i-data DADA2_full/CHBoral_PairedEndSeq_demux-full-L390-filtered-seqs.qza \
  --o-visualization DADA2_full/CHBoral_PairedEndSeq_demux-full-L390-filtered-seqs.qzv



Saved FeatureData[Sequence] to: DADA2_full/CHBoral_PairedEndSeq_demux-full-L390-filtered-seqs.qza
Saved Visualization to: DADA2_full/CHBoral_PairedEndSeq_demux-full-L390-filtered-seqs.qzv


In [7]:
'''View dada2 stats for demultiplexed seqs'''

!qiime tools view DADA2_full/CHBoral_PairedEndSeq_demux-full-L390-filtered-seqs.qzv


Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [14]:
'''Filter feature table for features based on filtered seqs'''
!qiime feature-table filter-features \
  --i-table DADA2_full/CHBoral_PairedEndSeq_demux-full-table.qza \
  --m-metadata-file DADA2_full/CHBoral_PairedEndSeq_demux-full-L390-filtered-seqs.qza \
  --o-filtered-table DADA2_full/CHBoral_PairedEndSeq_demux-full-L390-filtered-seqs-table.qza     


Saved FeatureTable[Frequency] to: DADA2_full/CHBoral_PairedEndSeq_demux-full-L390-filtered-seqs-table.qza


In [15]:
'''Get feature summary for filtered table'''

# Generates table with feature frequency per sample and frequency per feature
!qiime feature-table summarize \
  --i-table DADA2_full/CHBoral_PairedEndSeq_demux-full-L390-filtered-seqs-table.qza \
  --o-visualization DADA2_full/CHBoral_PairedEndSeq_demux-full-L390-filtered-seqs-table.qzv \
  --m-sample-metadata-file mappingfile_NileshMehta_with_Metadata_062715_corrected.txt


Saved Visualization to: DADA2_full/CHBoral_PairedEndSeq_demux-full-L390-filtered-seqs-table.qzv


In [16]:
'''View filtered feature tables'''

!qiime tools view DADA2_full/CHBoral_PairedEndSeq_demux-full-L390-filtered-seqs-table.qzv



Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>